In [2]:
!pip install altair

   ---------------------------------------- 857.8/857.8 kB 4.2 MB/s eta 0:00:00


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import requests #to read the URLs
import numpy as np
import altair as alt #visualziation package




c:\Users\13980611\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [4]:
df = pd.read_csv("neg_comet.csv")

df.head()


,Period,Type,Code,Value
0,NonCOVID,General,respect to patient,16.28
1,NonCOVID,General,nurse/nurse aide,24.61
2,NonCOVID,General,dietary/service,51.98
3,COVID,General,information/education,40.75
4,COVID,General,families/friends,50.11


In [5]:
type_include = ['Emergency', 'General', 'Maternity', 'Inpatient']
code_include = ['access/coord of care', 'emergency', 'radiology', 'discharge']
df = df[df['Value'].notna()]
df = df.query('Type in @type_include and Code in @code_include')

In [4]:
df = pd.read_csv("https://raw.githubusercontent.com/HIVE-UofT/Health-Data-Visualization/main/Lecture07/13100098.csv")
geo_exclude = ['Canada (excluding territories)', 'Quebec']
df = df.rename(columns={'Age group': 'age_group'})
df['age_group'] = df['age_group'].replace({'18 to 34 years':'18-34 yrs', '35 to 49 years':'35-49 yrs', '50 to 64 years':'50-64 yrs', 'Total, 12 years and over': 'Total', '65 years and over':'>=65 yrs'})
df = df.query(' GEO not in @geo_exclude and Characteristics=="Percent" and Sex=="Both sexes" and Indicators == "Suicidal thoughts (15 years and over)" and age_group!="12 to 17 years" and VALUE>0')
df = df[['REF_DATE', 'GEO', 'Indicators', 'VALUE', 'Sex','age_group']]
df = df[df['VALUE'].notna()]

In [6]:
df = pd.read_csv("neg_comet.csv")

df['Type'] = df['Type'].astype(str).str.strip()
df['Period'] = df['Period'].astype(str).str.strip()
df['Code'] = df['Code'].astype(str).str.strip()
df['Value'] = df['Value'].astype(str).str.strip()
df = df[df['Value'].notna()]
df['Code'] = df['Code'].replace({'information/education':'I/E', 'medication/prescription':'M/P', 'infection prevention & control': 'IC', 'families/friends': 'F/F', 'parking/transport': 'P/T', 'social services': 'SS', 'laboratory': 'Lab', 'radiology': 'Rad'})

df['Period'] = df['Period'].replace({'COVID': 'C', 'NonCOVID': 'NC'})

code_exclude = ['cardiology', 'positive recognition', 'respect to patient', 'religion', 'nurse/nurse aide', 'general comment', 'icu/ccu', 'housekeeping/room', 'physical comfort', 'emotional support', 'emergency', 'dietary/service', 'discharge', 'access/coord of care', 'admit/registration', 'billing/accounting', 'continuity/transition']
#code_exclude = ['dietary/service', 'emergency', 'access/coord of care', 'admit/registration', 'positive recognition', 'billing/accounting', 'general comment', 'respect to patient', 'religion', 'discharge', 'radiology', 'laboratory', 'parking/transport', 'infection prevention & control', 'cardiology', 'icu/ccu']
type_exclude = ['Rehabilitation', 'Service Alert', 'Outpatient', 'General', 'Maternity']

df = df.query('Code not in @code_exclude and Type not in @type_exclude')

df.head()


,Period,Type,Code,Value
98,NC,Emergency,P/T,65.62
100,NC,Emergency,Rad,53.35
106,C,Emergency,Rad,44.41
109,NC,Emergency,I/E,50.0
110,NC,Emergency,Lab,50.0


In [7]:
fig = (
    alt.Chart(df)
    .transform_pivot("Period", value="Value", groupby=["Type", "Code"])
    .transform_fold(["NC", "C"], as_=["Period", "Value"])  # Changed from "NonCOVID" and "COVID" to "NC" and "C"
    .transform_calculate(calculate="datum['C'] - datum['NC']", as_="delta")
    .mark_trail()
    .encode(
        x=alt.X('Period:N', title=None),
        y=alt.Y('Type:N', title=''),
        size=alt.Size('Value:Q', scale=alt.Scale(range=[0, 10]), legend=alt.Legend(title='Value', orient='top')),
        color=alt.Color('delta:Q', scale=alt.Scale(domainMid=0, scheme='redblue', reverse=True), legend=alt.Legend(title='Delta', orient='top')),
        tooltip=alt.Tooltip(['Period:N', 'Value:Q']),
        column=alt.Column(
            'Code:N', 
            title='', 
            header=alt.Header(
                labelAlign='left', 
                labelAnchor= 'middle'
            )
        )
    )
    .configure_view(stroke=None)
    .configure_legend(labelFontSize=20, titleFontSize=18)
    .configure_title(fontSize=20)
    .configure_axis(labelFontSize=20, titleFontSize=18)
    .configure_header(labelFontSize=18)
    .properties(title='', width=70, height=380)
)

In [8]:
fig

alt.Chart(...)

: 